In [ ]:
# geojson to raster
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

gdf = gpd.read_file("grid.geojson")

xmin, ymin, xmax, ymax = gdf.total_bounds
pixel_size = 500
width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

raster = rasterize(((geom, 1) for geom in gdf.geometry),
                   out_shape=(height, width),
                   transform=transform,
                   fill=0,
                   dtype="uint8")

with rasterio.open("grid.tif", "w", driver="GTiff", height=height, width=width,
                   count=1, dtype="uint8", crs=gdf.crs, transform=transform) as dst:
    dst.write(raster, 1)

print("Raster saved: grid.tif")

In [ ]:
# csv to shp
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

csv_file = "Power_plants.csv"
df = pd.read_csv(csv_file)

required_columns = {"Plant", "Capacity_GW", "Type", "Status", "longitude", "latitude"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

df["geometry"] = df.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

gdf = gdf.rename(columns={"Capacity_GW": "Cap_GW"})

gdf = gdf[["Plant", "Cap_GW", "Type", "Status", "geometry"]]

output_shp = "Powerplant.shp"
gdf.to_file(output_shp, driver="ESRI Shapefile")

print(f"Shapefile successfully saved to: {output_shp}")

In [ ]:
# Output raster with value 1 pixels 
import rasterio
import numpy as np

input_raster = "grid.tif"
output_raster = "grid1.tif"

with rasterio.open(input_raster) as src:
    data = src.read(1)
    
    masked_data = np.where(data == 1, 1, src.nodata)
    
    meta = src.meta.copy()
    meta.update(dtype="float32")

with rasterio.open(output_raster, "w", **meta) as dst:
    dst.write(masked_data, 1)

print(f"Raster with value 1 pixels saved to: {output_raster}")

In [ ]:
# Filter Features by a Specific Field in a Shapefile
import geopandas as gpd

input_shp = "gis_osm_roads_free_1.shp"  
output_shp = "Road_primary_1.shp"  

gdf = gpd.read_file(input_shp)

if "fclass" not in gdf.columns:
    raise ValueError("'fclass' field not found. Check the input file attributes.")

gdf_filtered = gdf[gdf["fclass"] == "primary"]

if gdf_filtered.empty:
    print("No features found with 'fclass' = 'primary'.")
else:
    print(f"Found {len(gdf_filtered)} features.")
    gdf_filtered.to_file(output_shp, driver="ESRI Shapefile")
    print(f"Filtered data saved to: {output_shp}")

In [ ]:
# shp to tif
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin

input_shp = "idn_admbnda_adm0_bps_20200401.shp"
output_raster = "Boundary.tif"

gdf = gpd.read_file(input_shp)
if gdf.empty:
    raise ValueError("Shapefile contains no valid geometries.")

xmin, ymin, xmax, ymax = gdf.total_bounds
pixel_size = 0.1
width, height = int((xmax - xmin) / pixel_size), int((ymax - ymin) / pixel_size)
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

shapes = ((geom, 1) for geom in gdf.geometry)
raster_data = rasterize(shapes, out_shape=(height, width), transform=transform, fill=0, dtype="float32")

with rasterio.open(output_raster, "w", driver="GTiff", height=height, width=width, count=1,
                   dtype="float32", crs=gdf.crs, transform=transform) as dst:
    dst.write(raster_data, 1)

print(f"Shapefile successfully converted to raster: {output_raster}")

In [ ]:
# merge shp
import geopandas as gpd
import pandas as pd

input_shps = [
    "shapefile1.shp",
    "shapefile2.shp",
    "shapefile3.shp",
]
output_shp = "merged_shapefile.shp"

gdfs = [gpd.read_file(shp) for shp in input_shps]
for i, gdf in enumerate(gdfs):
    print(f"Loaded {input_shps[i]}: {len(gdf)} features, CRS: {gdf.crs}")

merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

if len(set(gdf.crs for gdf in gdfs)) > 1:
    raise ValueError("Input shapefiles have inconsistent CRS. Please standardize before merging.")

merged_gdf.to_file(output_shp)
print(f"Merging complete. Output saved to: {output_shp}")

In [ ]:
# merge tif
import rasterio
from rasterio.merge import merge

input_tiffs = [
    "PRoad_1.tif", "PRoad_2.tif", "PRoad_3.tif",
    "PRoad_4.tif", "PRoad_5.tif", "PRoad_6.tif", "PRoad_7.tif"
]
output_tiff = "PRoad.tif"

src_files = [rasterio.open(fp) for fp in input_tiffs]

mosaic, out_transform = merge(src_files)

out_meta = src_files[0].meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform,
    "crs": src_files[0].crs
})

with rasterio.open(output_tiff, "w", **out_meta) as dest:
    dest.write(mosaic)

for src in src_files:
    src.close()

print(f"TIFF files successfully merged and saved as: {output_tiff}")

In [ ]:
# nc to tif
import xarray as xr
import rasterio
from rasterio.transform import from_origin
import numpy as np

input_nc = "C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.area-subset.6.1.95.-11.141.1.nc"
output_tiff = "landuse.tif"

nc_data = xr.open_dataset(input_nc)

if 'lccs_class' not in nc_data.data_vars:
    raise ValueError("'lccs_class' not found in the .nc file. Check the file contents.")

lccs_data = nc_data['lccs_class'].values
print(f"Original lccs_class data shape: {lccs_data.shape}")

if len(lccs_data.shape) == 3:  # (time, height, width)
    lccs_data_2d = lccs_data[0, :, :]
elif len(lccs_data.shape) == 2:
    lccs_data_2d = lccs_data
else:
    raise ValueError(f"Unsupported lccs_data dimensions: {lccs_data.shape}")

print(f"Processed lccs_class data shape: {lccs_data_2d.shape}")

lccs_data_2d = np.nan_to_num(lccs_data_2d, nan=-9999).astype("float32")

lat, lon = nc_data['lat'].values, nc_data['lon'].values

transform = from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[0] - lat[1])

with rasterio.open(
    output_tiff, 'w', driver='GTiff',
    height=lccs_data_2d.shape[0], width=lccs_data_2d.shape[1],
    count=1, dtype="float32",
    crs="EPSG:4326",  # Assuming WGS84 projection
    transform=transform,
) as dst:
    dst.write(lccs_data_2d, 1)

print(f"lccs_class data successfully saved as raster: {output_tiff}")

In [ ]:
# calculate Slope
import rasterio
import numpy as np
from scipy.ndimage import sobel

input_file = "DEM_trans.tif" 
output_file = "Slope_Angle.tif"

def calculate_slope(dem_data, x_res, y_res):
    """
    Calculate slope (in degrees) from DEM data.
    
    :param dem_data: DEM data array
    :param x_res: Raster resolution in x direction
    :param y_res: Raster resolution in y direction
    :return: Slope array (in degrees)
    """
    dem_data = np.where((dem_data < -500) | (dem_data > 9000), np.nan, dem_data)

    if x_res <= 0 or y_res <= 0:
        raise ValueError("Invalid raster resolution: x_res or y_res must be positive.")

    dz_dx = sobel(dem_data, axis=1) / (8 * x_res)
    dz_dy = sobel(dem_data, axis=0) / (8 * y_res)
    
    slope_degrees = np.degrees(np.arctan(np.sqrt(dz_dx**2 + dz_dy**2)))

    return np.where(np.isnan(slope_degrees), 0, slope_degrees)

def process_dem_to_slope(input_file, output_file):
    """
    Compute slope from DEM and save as a new raster file.
    
    :param input_file: Path to input DEM file
    :param output_file: Path to output slope raster file
    """
    with rasterio.open(input_file) as src:
        dem_data = src.read(1)
        profile = src.profile.copy()
        x_res, y_res = abs(src.transform[0]), abs(src.transform[4])

        slope_data = calculate_slope(dem_data, x_res, y_res)

        profile.update(dtype=rasterio.float32, count=1, nodata=0)

        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(slope_data.astype(np.float32), 1)

    print(f"Slope calculation completed. Output saved to {output_file}")

process_dem_to_slope(input_file, output_file)

In [ ]:
# calculate distance
import os
import numpy as np
import rasterio
from scipy.ndimage import distance_transform_edt

input_file = "Urban_Reprojected.tif"
output_file = "Urban_Distance.tif"

def calculate_distance_to_nonzero(input_file, output_file, resolution=None):
    """
    Computes the Euclidean distance to the nearest nonzero pixel in a raster and saves the result.
    
    :param input_file: Path to the input raster file
    :param output_file: Path to save the output distance raster
    :param resolution: Target resolution (same unit as CRS, defaults to input resolution)
    """
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")

    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with rasterio.open(input_file) as src:
        src_data = src.read(1)
        transform = src.transform
        profile = src.profile.copy()
     
        resolution = resolution or abs(transform[0])

        mask = src_data > 0
        print(f"Nonzero pixel count: {np.sum(mask)}")

        if not mask.any():
            raise ValueError("No nonzero pixels found in the input raster.")

        distances = distance_transform_edt(~mask) * resolution

        distances[mask] = 0

        profile.update({
            "dtype": rasterio.float32,
            "compress": "lzw",
            "nodata": None,
        })

        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(distances.astype(np.float32), 1)

    print(f"Distance computation completed. Output saved to: {output_file}")

calculate_distance_to_nonzero(input_file, output_file)